$$
\kappa(u) = \left(\frac{3u(1-u)}{3u^3 + (1-u)^3}\right)^2  + \mu
$$
$$
\begin{equation}
\left\{
	\begin{aligned}
	&-\nabla\cdot(\kappa(u)\nabla u) = f, &x \in \Omega, \\
	&u = 0, &x \in \partial\Omega
\end{aligned}
\right.
\end{equation}
$$

$$
	f(x; i^\star) = 100 \exp(-50 ||x - x_{c, i^\star}||_2^2)\mathbf{1}_{\Omega_{i^\star}}(x)
$$

In [ ]:
from utils import fd_solve_nlinear
import numpy as np

GridSize = 256
area = ((0, 0), (1, 1))
mu = 0.1

# fd_solve_nlinear(GridSize, area, 0.1, (0.1, 0.1))
ans = []
for i in np.arange(0.1, 1, 0.1):
	for j in np.arange(0.1, 1, 0.1):
		center = (i, j)
		ans.append(fd_solve_nlinear(GridSize, area, mu, center,))

np.stack(ans)
np.save(f'./DLdata/nlinear/{GridSize}/U.npy', ans)

In [ ]:
from FVM.src.UniformICD import UniformFVM
from FVM.src.Problem.test_uniform import *
from FVM.src.utils import show_errors

# Test zxp case !!!
case_ns = list(range(10))
print(f"|{'Method': ^8s}|{'Case ID': ^8s}|{'Mesh Type': ^18s}|{'hmesh': ^11s}|{'error_max': ^11s}|{'ratio': ^7s}|{'error_l2': ^11s}|{'ratio': ^7s}|{'error_h1': ^11s}|{'ratio': ^7s}|")
print(f'|{":--:": ^8s}|{":--:": ^8s}|{":--:": ^18s}|{":--:": ^11s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|{":--:": ^11s}|{":--:": ^7s}|')
for case_n in case_ns:
	mesh_type = 'quadrangle_uniform'
		# triangle_classic triangle_random triangle_uniform triangle_kershaw
		# quadrangle_uniform quadrangle_random  quadrangle_kershaw
	n_levels = 5
	hmeshes = []
	errors_max = []
	errors_l2 = []
	errors_h1 = []

	for i in range(n_levels):
		N = 2**(i+3)
		problem = zxpProblem(N, N, ((0, 0), (1, 1)), case_n, 1e-9)
		solver = UniformFVM(((0, 0), (1, 1)), N, N, problem)
		solver.solve(solver_name=None)
		hmesh, error_max, error_l2, error_h1 = solver.compute_errors()

		hmeshes.append(hmesh)
		errors_max.append(error_max)
		errors_l2.append(error_l2)
		errors_h1.append(error_h1)

	for s in show_errors(hmeshes, errors_max, errors_l2, errors_h1):
		print(f'|{"ICD": ^8s}|{case_n: ^8}|{mesh_type: ^18s}|' + s)

In [ ]:
from Problems.NonlinearProblem import *
from plots import my_plot
from UniformICD import UniformFVM

def fvm_solve_nlinear(GridSize, area, mu, center=(0.5, 0.5), 
					Picard_maxiter=1000, eps=1e-7):
	u0 = np.random.rand(GridSize, GridSize)
	problem = NonLinearProblem(None, center, N, area)
	solver = UniformFVM(area, N, N, problem)
	solver.solve(solver_name = None)
	u0,  b = solver.ua, solver.b

	for i in range(Picard_maxiter):
		problem = NonLinearProblem(kappa(u0.reshape(GridSize, GridSize), mu), center, N, area)

		solver = UniformFVM(area, N, N, problem)
		solver.solve(solver_name = None)
		newA = solver.A.tocsr()
		newu = solver.ua

		# delta = ((newu - u0)**2 * h**2).sum()
		delta = np.linalg.norm(newu - u0)
		# error = ((newA @ u0 - b)**2 * h**2).sum()
		error = np.linalg.norm(newA @ u0 - b)
		print(f"Itr: {i}\t Delta: {delta:.3e}\t Error: {error:.3e}\t")

		if delta <= eps or error <= eps:
			u = newu.reshape(GridSize, GridSize)
			break
		else:
			u0, A0 = newu, newA
	return u


N = 128
mu = 0.1
center = (0.1, 0.1)
area = ((0, 0), (1, 1))

solution = fvm_solve_nlinear(N, area, mu)

In [ ]:
from fit_multiCOF_ALL import C1Ds
import torch
from torch.utils.data import DataLoader
from Generators import JacBatched

device = 'cpu'
dtype = torch.float

ds = C1Ds(0, 10, ((0, 0), (1, 1)), 96, dtype, device)
# for data, cof, A, b, u in ds:
#     print(data.shape)
#     print(cof.shape)
#     print(A.shape)
#     print(b.shape)
#     print(u.shape)
#     break

dl = DataLoader(ds, batch_size=2)
for data, cof, A, b, u in dl:
	# print(data.shape)
	# print(cof.shape)
	# print(A.shape)
	# print(b.shape)
	# print(u.shape)
	generator = JacBatched(A, dtype, device)
	# print(generator.M.shape)
	# print(generator.invD.shape)
	y = generator(torch.rand(2, 1, 96, 96), b[..., None], 5)
	break




In [1]:
from Generators import CGBatched
import torch

N = 16
N2 = N**2
batch_size = 3
A = torch.rand(batch_size, N2, N2)
b = torch.rand(batch_size, N2, 1)
u = torch.rand(batch_size, 1, N, N)
generator = CGBatched(A, 'cpu', torch.float)
v = generator(u, b, 5)


error: 3.417e-02
error: 5.852e+00
error: 3.639e+03
error: -1.395e+06
error: -2.632e+07


In [2]:
from gen_data import ChipsDataGenerator
generator = ChipsDataGenerator(10, ((0, 0), (0.1, 0.1)), 0.001, 0.001)
generator.generate(
    'DLdata/nouse.csv',
    'DLdata/heat',
    256,
    solve=False
)

100%|██████████| 10/10 [00:00<00:00, 20.89it/s]
